# Démonstration : Hachage Sécurisé de Mots de Passe

Ce notebook illustre les techniques de stockage sécurisé de mots de passe :
- Hachage simple (SHA-256)
- Hachage avec sel (salt)
- PBKDF2
- bcrypt

In [ ]:
import hashlib
import os
import time
from hashlib import pbkdf2_hmac

# Installer bcrypt si nécessaire
try:
    import bcrypt
except ImportError:
    !pip install bcrypt
    import bcrypt

## 1. Hachage simple (INSÉCURISÉ)

In [ ]:
password = "monmotdepasse123"

# Hachage SHA-256 simple
hash_simple = hashlib.sha256(password.encode()).hexdigest()
print(f"Hash SHA-256 simple : {hash_simple}")

# Problème : Deux utilisateurs avec le même mot de passe auront le même hash
password2 = "monmotdepasse123"
hash_simple2 = hashlib.sha256(password2.encode()).hexdigest()
print(f"\nMême mot de passe -> même hash : {hash_simple == hash_simple2}")
print("❌ Vulnérable aux rainbow tables !")

## 2. Hachage avec sel (salt)

In [ ]:
def hash_password_with_salt(password):
    # Générer un sel aléatoire de 32 bytes
    salt = os.urandom(32)
    
    # Hacher password + salt
    pwd_hash = hashlib.sha256(salt + password.encode()).hexdigest()
    
    # Retourner sel + hash (les deux sont nécessaires pour vérification)
    return salt.hex(), pwd_hash

def verify_password(password, salt_hex, stored_hash):
    salt = bytes.fromhex(salt_hex)
    pwd_hash = hashlib.sha256(salt + password.encode()).hexdigest()
    return pwd_hash == stored_hash

# Test
password = "monmotdepasse123"
salt, hash1 = hash_password_with_salt(password)
print(f"Sel : {salt[:32]}...")
print(f"Hash : {hash1}")

# Même mot de passe, sel différent -> hash différent
salt2, hash2 = hash_password_with_salt(password)
print(f"\nMême mot de passe, sel différent :")
print(f"Hash 1 : {hash1}")
print(f"Hash 2 : {hash2}")
print(f"Différents : {hash1 != hash2}")
print("✅ Les rainbow tables sont inutiles !")

# Vérification
print(f"\nVérification avec bon mot de passe : {verify_password('monmotdepasse123', salt, hash1)}")
print(f"Vérification avec mauvais mot de passe : {verify_password('mauvais', salt, hash1)}")

## 3. PBKDF2 (Password-Based Key Derivation Function)

In [ ]:
def hash_pbkdf2(password, iterations=100000):
    salt = os.urandom(32)
    pwd_hash = pbkdf2_hmac('sha256', password.encode(), salt, iterations)
    return salt.hex(), pwd_hash.hex()

def verify_pbkdf2(password, salt_hex, stored_hash, iterations=100000):
    salt = bytes.fromhex(salt_hex)
    pwd_hash = pbkdf2_hmac('sha256', password.encode(), salt, iterations)
    return pwd_hash.hex() == stored_hash

# Test de performance
print("Benchmark PBKDF2 avec différents nombres d'itérations :")
for iterations in [10000, 100000, 500000]:
    start = time.time()
    salt, hash_val = hash_pbkdf2("test", iterations)
    elapsed = time.time() - start
    print(f"  {iterations:6d} itérations : {elapsed:.4f}s")

print("\n✅ Plus d'itérations = plus lent = plus difficile à bruteforcer")

## 4. bcrypt (Recommandé)

In [ ]:
def hash_bcrypt(password, cost=12):
    # bcrypt gère automatiquement le sel
    pwd_bytes = password.encode('utf-8')
    hashed = bcrypt.hashpw(pwd_bytes, bcrypt.gensalt(rounds=cost))
    return hashed.decode('utf-8')

def verify_bcrypt(password, stored_hash):
    pwd_bytes = password.encode('utf-8')
    hash_bytes = stored_hash.encode('utf-8')
    return bcrypt.checkpw(pwd_bytes, hash_bytes)

# Test
password = "SecureP@ssw0rd!"
hashed = hash_bcrypt(password)
print(f"Hash bcrypt : {hashed}")
print(f"\nLongueur : {len(hashed)} caractères")
print("Note : Le hash contient le sel ET le coût")

# Vérification
print(f"\nVérification correct password : {verify_bcrypt(password, hashed)}")
print(f"Vérification wrong password : {verify_bcrypt('wrong', hashed)}")

# Benchmark
print("\nBenchmark bcrypt avec différents coûts :")
for cost in [10, 12, 14]:
    start = time.time()
    hash_bcrypt("test", cost)
    elapsed = time.time() - start
    iterations = 2**cost
    print(f"  Cost {cost} ({iterations:5d} itérations) : {elapsed:.4f}s")

print("\n✅ bcrypt est actuellement recommandé pour le hachage de mots de passe")

## 5. Comparaison des méthodes

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Méthode': ['SHA-256 simple', 'SHA-256 + sel', 'PBKDF2 (100k iter)', 'bcrypt (cost 12)'],
    'Sécurité': ['❌ Faible', '⚠️ Moyenne', '✅ Bonne', '✅ Excellente'],
    'Rainbow tables': ['Vulnérable', 'Résistant', 'Résistant', 'Résistant'],
    'Brute force': ['Rapide', 'Rapide', 'Lent', 'Très lent'],
    'Recommandation': ['❌ Non', '⚠️ Insuffisant', '✅ OK', '✅ Recommandé']
})

print(comparison.to_string(index=False))

print("\n📝 Bonnes pratiques :")
print("  1. Toujours utiliser un sel unique par utilisateur")
print("  2. Utiliser bcrypt ou Argon2 (pas SHA-256 seul)")
print("  3. Choisir un coût adapté (bcrypt cost 12-14)")
print("  4. Mettre à jour les hashs si l'algorithme est compromis")

## Exercice : Attaque par dictionnaire

Simulons une attaque par dictionnaire sur différentes méthodes.

In [ ]:
# Dictionnaire de mots de passe courants
common_passwords = [
    "123456", "password", "123456789", "12345678", "12345",
    "qwerty", "abc123", "monkey", "letmein", "trustno1"
]

# Hash SHA-256 simple à craquer
target_hash_sha256 = hashlib.sha256("password".encode()).hexdigest()

print("Attaque par dictionnaire sur SHA-256 simple :")
start = time.time()
for pwd in common_passwords:
    if hashlib.sha256(pwd.encode()).hexdigest() == target_hash_sha256:
        print(f"✅ Mot de passe trouvé : {pwd}")
        break
elapsed = time.time() - start
print(f"Temps : {elapsed:.6f}s pour {len(common_passwords)} essais\n")

# Attaque sur bcrypt (beaucoup plus lent)
target_hash_bcrypt = hash_bcrypt("password")
print("Attaque par dictionnaire sur bcrypt :")
start = time.time()
for pwd in common_passwords:
    if verify_bcrypt(pwd, target_hash_bcrypt):
        print(f"✅ Mot de passe trouvé : {pwd}")
        break
elapsed = time.time() - start
print(f"Temps : {elapsed:.6f}s pour {len(common_passwords)} essais")

print("\n💡 Conclusion : bcrypt ralentit considérablement les attaques par force brute")